In [3]:
import copy
import scipy
from scipy import optimize

In [79]:
def simplex(A, b, c):
    return scipy.optimize.linprog(c, A_ub=A, b_ub=b)

In [94]:
def branch_and_bound(a, b, c):
    programs = [[c, a, b]]
    sols = []
    num = 0
    
    while (programs != []):
        curr_program = programs.pop(0)
        curr_c, curr_a, curr_b = curr_program
        
        res = simplex(
            curr_a, 
            curr_b, 
            curr_c
        )
        z = res.fun
        x = res.x
        
        if res.status == 2: continue
            
        non_int_sols = []
        
        for i in range(len(x)):
            if x[i] != int(x[i]):
                non_int_sols.append(x[i])
                
        if len(non_int_sols) == 0:
            sols.append([x, z])
            continue
        
        max_non_int_sol_index = list(x).index(max(non_int_sols))
        
        new_constraint = [0] * len(x)
                
        new_constraint[max_non_int_sol_index] = 1
                
        programs.append([
            curr_c,
            curr_a + [copy.deepcopy(new_constraint)],
            curr_b + [int(x[max_non_int_sol_index])],
        ])
                
        new_constraint[max_non_int_sol_index] = -1
                
        programs.append([
            curr_c,
            curr_a + [copy.deepcopy(new_constraint)],
            curr_b + [-(int(x[max_non_int_sol_index]) + 1)],
        ])
                
    return sols

In [95]:
c = [1, -2]
a = [[-4, 6], [4, 1]]
b = [3, 4]

print(branch_and_bound(a, b, c))

[[array([1., 0.]), 1.0], [array([0., 0.]), 0.0]]


In [91]:
simplex([[-4, 6], [4, 1], [1, 0]], [3, 4, 0], [1, -2])

     con: array([], dtype=float64)
     fun: -1.0
 message: 'Optimization terminated successfully.'
     nit: 2
   slack: array([0. , 3.5, 0. ])
  status: 0
 success: True
       x: array([0. , 0.5])